In [4]:
! pip install ..

Processing c:\users\acmiy\onedrive\documents\cs229\cs229-f19-wiki-forecast
  Created wheel for wiki-forecast: filename=wiki_forecast-0.3.3-cp36-none-any.whl size=23938 sha256=d171b34fbd9afe6713db2dbf7feecb82d86ff7db1fa65adb83fc8170e0a614f6
  Stored in directory: C:\Users\acmiy\AppData\Local\pip\Cache\wheels\55\63\bc\107d7790eee90396286fcf797d47bb71ad21153db264e762b8
Successfully built wiki-forecast
  Found existing installation: wiki-forecast 0.2
    Uninstalling wiki-forecast-0.2:
      Successfully uninstalled wiki-forecast-0.2


In [13]:
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from functools import partial
import itertools
import warnings
import numpy as np

import scipy.stats as stats
from sklearn.metrics import make_scorer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor

#from wikicast.data import create_dataset
from wikicast.data import create_dataset, mape, rmse
from wikicast.util import (
    run_ablation,
    run_train_predict,
    summarize,
    write_search_results,
    plot_learning_curve,
)

N_CUTS = 8
START_DATE = datetime.strptime("2018-01-01", "%Y-%m-%d")
END_DATE = datetime.strptime("2019-09-01", "%Y-%m-%d")

META = ["partition_id", "title"]
DEGREE = ["degree", "inDegree", "outDegree"]
SIGN = [f"sign_{i}" for i in range(N_CUTS)]
VECTOR = [f"fiedler_{i}" for i in range(N_CUTS)]
DATES = [(START_DATE + timedelta(t)).strftime("%Y-%m-%d")
         for t in range((END_DATE-START_DATE).days)]

SCHEMA = META + DEGREE + SIGN + VECTOR + DATES


data = pd.read_parquet(glob.glob("../data/design_matrix/sample_6_8_50/*.parquet")[0]).fillna(0)
data = data.sample(int(1e4))
window_size = 7
num_windows = 150 // window_size

train, validate, test = create_dataset(data[DATES], window_size, num_windows)
scaler = StandardScaler()
features = [
    data[DEGREE].values,
    data[SIGN].values,
    scaler.fit_transform(data[VECTOR].values),
]
features_dict = dict(zip(["degree", "sign", "vector"], features))

output = "../data/results/experiments_keras"

C:\Users\acmiy\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [14]:
print(f"train shape: {train.shape}")
print(f"validate shape: {validate.shape}")
print(f"test shape: {test.shape}")

results = [
    summarize("persistence", test, validate),
    summarize("mean", test, (np.ones(test.shape).T * validate.mean(axis=1)).T),
]

pred = run_train_predict(Ridge(alpha=0), train, validate, test, [])
results += [summarize("linear regression", test, pred)]

scoring = {
    "rmse": make_scorer(rmse, greater_is_better=False),
    "mape": make_scorer(mape, greater_is_better=False),
}

ridge = Ridge()
params = dict(alpha=stats.reciprocal(a=1e5, b=1e8))
search_ridge = RandomizedSearchCV(
    estimator=ridge,
    param_distributions=params,
    scoring=scoring,
    refit="rmse",
    cv=5,
    n_iter=10,
    return_train_score=False,
)
results += run_ablation(
    "ridge regression", search_ridge, train, validate, test, features_dict
)
write_search_results(search_ridge, f"{output}/ridge-random.csv")

train shape: (10000, 133)
validate shape: (10000, 7)
test shape: (10000, 7)
   mean_test_rmse  mean_test_mape  rank_test_rmse  rank_test_mape  \
7     -342.841425      -32.982091               1               1   
9     -413.101688      -39.422721               2               2   
6     -423.402311      -40.554082               3               3   
2     -431.983429      -41.554732               4               4   
1     -449.422517      -43.715286               5               5   
0     -455.804391      -44.530029               6               6   
5     -457.084214      -44.695040               7               7   
4     -458.862794      -44.926142               8               8   
8     -459.645697      -45.030439               9               9   
3     -459.858631      -45.058837              10              10   

   mean_fit_time  param_alpha  
7       0.119410  6.92051e+07  
9       0.116353  1.50622e+07  
6       0.114905  1.08137e+07  
2       0.115877  7.72252e+06  
1   

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,...,std_test_rmse,rank_test_rmse,split0_test_mape,split1_test_mape,split2_test_mape,split3_test_mape,split4_test_mape,mean_test_mape,std_test_mape,rank_test_mape
7,0.119410,0.005442,0.019816,0.000982,6.92051e+07,{'alpha': 69205084.15138026},-570.200894,-246.428019,-179.693676,-370.783001,...,132.969125,1,-39.058479,-33.955959,-28.385941,-34.918597,-28.591481,-32.982091,4.050260,1
9,0.116353,0.002687,0.019646,0.000460,1.50622e+07,{'alpha': 15062174.263144456},-693.003366,-339.193698,-247.301869,-427.642368,...,151.330250,2,-45.995265,-41.758014,-34.118940,-42.077906,-33.163481,-39.422721,4.960043,2
6,0.114905,0.001602,0.019412,0.000491,1.08137e+07,{'alpha': 10813665.987996925},-704.875177,-351.543945,-259.959951,-439.944184,...,151.862230,3,-47.082627,-43.006757,-35.208104,-43.384296,-34.088628,-40.554082,5.040421,3
2,0.115877,0.002948,0.020016,0.000007,7.72252e+06,{'alpha': 7722515.605369685},-713.777186,-361.246771,-270.701053,-451.393778,...,152.043899,4,-48.123020,-44.053119,-36.169831,-44.513375,-34.914313,-41.554732,5.123090,4
1,0.116308,0.003993,0.019948,0.000979,2.55554e+06,{'alpha': 2555542.2471496467},-729.473490,-378.985583,-292.169195,-478.820623,...,152.096857,5,-50.587548,-46.135549,-38.244396,-46.896781,-36.712155,-43.715286,5.332580,5
0,0.124464,0.005720,0.021020,0.001899,1.00559e+06,{'alpha': 1005585.9464866648},-734.540910,-384.752921,-299.580898,-490.478381,...,152.164225,6,-51.550066,-46.874607,-39.025229,-47.790369,-37.409875,-44.530029,5.411233,6
5,0.116305,0.001831,0.019619,0.000492,715375,{'alpha': 715375.2467786756},-735.519755,-385.861659,-301.022056,-492.930322,...,152.189231,7,-51.747493,-47.020023,-39.182261,-47.971339,-37.554084,-44.695040,5.426983,7
4,0.117498,0.004130,0.019618,0.000486,323616,{'alpha': 323615.71489109023},-736.859775,-387.373513,-302.997053,-496.405983,...,152.232244,8,-52.022669,-47.220785,-39.401422,-48.223381,-37.762454,-44.926142,5.446812,8
8,0.116906,0.003608,0.020020,0.000008,156295,{'alpha': 156294.78797084463},-737.440480,-388.022170,-303.856780,-497.969745,...,152.254795,9,-52.143476,-47.321919,-39.497852,-48.333655,-37.855291,-45.030439,5.456183,9
3,0.115610,0.001937,0.019478,0.000451,111900,{'alpha': 111899.9397404985},-737.596301,-388.192896,-304.090219,-498.404047,...,152.261533,10,-52.175751,-47.350989,-39.523567,-48.363256,-37.880624,-45.058837,5.458747,10


### Neural Net

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

def make_model(hidden_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers

    dense_layer_sizes: List of layer sizes.
        This list has one number for each layer
    filters: Number of convolutional filters in each convolutional layer
    kernel_size: Convolutional kernel size
    pool_size: Size of pooling area for max pooling
    '''

    model = Sequential()
    for layer_size in hidden_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    return model

NNRegressor = KerasRegressor(make_model, batch_size=32)

def best_nn_grid(params, output, **kwargs):
    print(f"running for {output}")
    nn = NNRegressor
    search = GridSearchCV(
        estimator=nn,
        param_grid=params,
        scoring=scoring,
        refit="rmse",
        cv=5,
        n_jobs=-1,
        return_train_score=False,
        **kwargs,
    )
    search.fit(np.hstack([train] + features), validate)
    write_search_results(search, output)
    return search

def best_nn_random(params, output, **kwargs):
    print(f"running for {output}")
    nn = NNRegressor
    search = RandomizedSearchCV(
        estimator=nn,
        param_distributions=params,
        scoring=scoring,
        refit="rmse",
        cv=5,
        n_jobs=-1,
        return_train_score=False,
        **kwargs,
    )
    search.fit(np.hstack([train] + features), validate)
    write_search_results(search, output)
    return search

In [ ]:
layers = [(16, 8, 8, 8)]
params = {
    "hidden_layer_sizes": layers
}
search = best_nn_grid(params, f"{output}/nn-grid-layers-best.csv")

running for ../data/results/experiments_keras/nn-grid-layers-best.csv


In [ ]:
%load_ext tensorboard